In [4]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import os
  CWD = '/content/drive/MyDrive/DataSources/SkinCare'
  os.chdir(CWD)
except:None
G_path = './Project_Data'

Mounted at /content/drive


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset as dset
#import torchvision.transforms.Compose
import numpy as np
from datetime import datetime	
import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import random_split
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
import pickle as pickle
from statistics import mean
import pandas as pd
pd.options.display.max_colwidth = 250
import sklearn as sk
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')
import features
from features.usefull_functions import *
from features.NETs import *
from features.Model_Training import *
device = None
try:
    c = torch.cuda.is_available()
    if c:
        print('devis: cuda')
        device = 'cuda'
    else:
        try :
            m = torch.backends.mps.is_available()
            if m:
                device = 'mps'
                print('devis: mps')  
        except:    
            device = 'cpu'
            print('devis: cpu')           
except:
    None

devis: cuda


In [6]:
Binary_classification = False
file = 'Input_DataSet_280x210' 
input_data , labels = pickle.load(open(G_path + '/06_Rescaled_DataSet/'+ file,'rb'))
file = 'Balanced_Test_Set_280x210' 
Evaluation_set = pickle.load(open(G_path + '/06_Rescaled_DataSet/' + file,'rb'))

if Binary_classification:
  labels = labels[1]
else:
  labels = labels[0]

In [7]:
labels[1][:10]

tensor([5, 5, 4, 5, 2, 1, 5, 2, 5, 5])

# **CNN Networks: Configuration**

In [8]:
net_list = [func for func in dir(CNN_Nets) if callable(getattr(CNN_Nets, func)) and not func.startswith("__")]
X_ = input_data[0][0:5]
for net in net_list:
    print(net)
    model_ = getattr(CNN_Nets,net)
    fc_features = model_().dimention_set(X_)
    print(fc_features)
    model_ = getattr(CNN_Nets,net)
    model_.fc_features = fc_features
    out = model_().forward(X_)
    setattr(CNN_Nets,net,model_)
    # print(out.shape)

Net1
202752
Net10
4480
Net11
1080
Net2
221952
Net3
12288
Net4
12288
Net5
56304
Net6
55488
Net7
384
Net8
384
Net8_a
384
Net8_a_binary
1536
Net8_b
1536
Net8_b_binary
1536
Net9
384


# **Metrics and Performance**

In [6]:


def plot_loss_accuracy(model_):
    epochs_X = [i for i in range(1, len(model_.Epochs_Train_loss)+1)]
    fig, axs = plt.subplots(1,2,figsize=(14,4))
    axs[0].plot(epochs_X , model_.Epochs_Train_loss , 'bo-', label='Train loss')
    axs[0].plot(epochs_X , model_.Epochs_Val_loss,'ro-', label='Validation loss')
    axs[0].plot(epochs_X , model_.Epochs_test_loss,'go-', label='Test loss')
    axs[0].set_xlabel("Epochs", fontsize = 12)
    axs[0].set_ylabel("Loss", fontsize = 12)
    axs[0].grid()
    axs[0].legend()
    axs[0].set_title('Train and Validation loss by epochs', fontsize = 14)
    axs[1].plot(epochs_X , model_.Epochs_Train_Acc , 'bo-', label='Train Accuracy')
    axs[1].plot(epochs_X , model_.Epochs_Val_Acc ,'ro-', label='Validation Accuracy')
    axs[1].plot(epochs_X , model_.Epochs_test_Acc ,'go-', label='Test Accuracy')
    axs[1].set_xlabel("Epochs", fontsize = 12)
    axs[1].set_ylabel("Accuracy", fontsize = 12)
    axs[1].grid()
    axs[1].legend()
    axs[1].set_title('Train and Validation Accuracy by epochs', fontsize = 14)
    plt.show()

def recall_specificity_precision(Y,Y_pred, weighted_avg):
    CM = sk.metrics.confusion_matrix(Y,Y_pred)
    FP = CM.sum(axis=0) - np.diag(CM) 
    FN = CM.sum(axis=1) - np.diag(CM)
    TP = np.diag(CM)
    TN = CM.sum() - (FP + FN + TP)
    weights = CM.sum(axis=1) / CM.sum() 
    Accuracy = np.nan_to_num((TP+TN)/(TP+FP+FN+TN) , nan=0)
    Recall = np.nan_to_num(TP/(TP+FN) , nan=0)
    Specificity = np.nan_to_num(TN/(TN+FP) , nan=0)
    Precision = np.nan_to_num(TP/(TP+FP) , nan=0)
    if weighted_avg:
        return round(sum(weights*Recall),3), round(sum(weights*Specificity),3), round(sum(weights*Precision),3)
    else:
        return round(mean(Recall),3), round(mean(Specificity),3), round(mean(Precision),3)

# **Define Training data**

In [9]:
X_train = input_data[0]
Y_train = labels[0]
X_valid = input_data[1]
Y_valid = labels[1]
X_test = input_data[2]
Y_test = labels[2]


label_freq = np.bincount(Y_train)
labels_weights = 1. / label_freq
weights = labels_weights[Y_train]
w_sampler = WeightedRandomSampler(weights, len(weights))

trainDataset = dset(X_train, Y_train)
validDataset = dset(X_valid, Y_valid)
testDataset = dset(X_test, Y_test)

# **Model by Grid**

In [11]:

Net = [CNN_Nets.Net8_b]
Drop = [0.25]
LR = [ 1.1e-3]
batch_size = [24]
Momentum = [0.77]
epochs = [2]
patience = [20]
weight_decay = [1e-3]
loss_func  =  [nn.CrossEntropyLoss]
opt_func = [torch.optim.SGD]


grid = {
    'Net' : Net
    ,'Drop' : Drop
    ,'LR' : LR
    ,'batch_size' : batch_size
    ,'Momentum' : Momentum
    ,'epochs' : epochs
    ,'patience': patience
    ,'weight_decay' :weight_decay
    ,'loss_func'  :loss_func
    ,'opt_func' : opt_func
}
params = sk.model_selection.ParameterGrid(grid)

In [11]:
print_epochs = True
save_results = False
save_models = False
cross_valid = False
nr_repeat =1

export_name = 'CNN_280x210_MultiClass' # {'280_210_','AutoEncoder_'}

Hyper_Details = pd.DataFrame(columns=['model_name','hyper_param','test_accuracy','valid_accuracy','train_accuracy','test_specificity','valid_specificity'
                                  ,'train_epoch_loss','train_epoch_acc','valid_epoch_loss','valid_epoch_acc','test_epoch_acc','test_epoch_loss','test_fpr','test_tpr','test_roc_auc','train_index','valid_index'])
date_hour = datetime.now().strftime("%d_%b%y_%H-%M") 
Max_train_Acc = 0  
Max_valid_Acc = 0 
Max_test_Acc = 0                  
Max_test_Recall = 0
Max_BLC_Test_Recall = 0
Max_test_overall_metric = 0
i = -1
pbar = tqdm(params)
for p in pbar:
    i += 1
    model_name =  export_name + 'Model' + str(i) + '_' + date_hour
    torch.cuda.empty_cache()
    Model_ = Model_Training_with_loader(**p,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset, testDataset=testDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=p)
    np.random.seed(0)
    random.seed(0)
    torch.manual_seed(0)
    Model_.train()

    train_accuracy = max(Model_.Epochs_Train_Acc)
    # valid_accuracy = max(Model_.Epochs_Val_Acc)
    # test_accuracy = max(Model_.Epochs_test_Acc)
    train_epoch_loss = Model_.Epochs_Train_loss
    train_epoch_acc = Model_.Epochs_Train_Acc
    valid_epoch_loss = Model_.Epochs_Val_loss
    valid_epoch_acc = Model_.Epochs_Val_Acc
    test_epoch_loss = Model_.Epochs_test_loss
    test_epoch_acc = Model_.Epochs_test_Acc


    model_ = Model_.model.eval().to('cpu')
    # test Avg. Sensitivity
    X = input_data[2]
    Y = labels[2]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    test_accuracy = sk.metrics.accuracy_score(Y, Y_pred )
    test_precision, test_recall, test_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')
    # Valid Avg. Sensitivity
    X = input_data[1]
    Y = labels[1]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    valid_accuracy = sk.metrics.accuracy_score(Y, Y_pred )
    valid_precision, valid_recall, valid_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')
    # Balanced test set Avg. Sensitivity
    X = Evaluation_set[0]
    Y = Evaluation_set[1]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    blc_test_precision, blc_test_recall, blc_test_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')

    test_overall_metric = (test_accuracy + test_recall + blc_test_recall ) / 3
    if test_overall_metric > Max_test_overall_metric: pickle.dump(model_ , open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  'Best_Grid_Model' +str(i) +'_' + date_hour, 'wb'))

    if train_accuracy > Max_train_Acc: Max_train_Acc = train_accuracy
    if valid_accuracy > Max_valid_Acc: Max_valid_Acc = valid_accuracy
    if test_accuracy > Max_test_Acc: Max_test_Acc = test_accuracy
    if test_recall > Max_test_Recall: Max_test_Recall = test_recall
    if blc_test_recall > Max_BLC_Test_Recall: Max_BLC_Test_Recall = blc_test_recall

    pbar.set_description(f'Max_Test_Recall: {Max_test_Recall:.1%}, Max_BLC_Test_Recall: {Max_BLC_Test_Recall:.1%}, Max_Tr_Acc: {Max_train_Acc:.1%}, Max_V_Acc: {Max_valid_Acc:.1%}, Max_Te_Acc: {Max_test_Acc:.1%} ')
    pbar.set_postfix_str(f'Train_Acc: {train_accuracy:.1%} , Valide_Acc: {valid_accuracy:.1%} , Test_Acc: {test_accuracy:.1%}')

    new_row = pd.Series({'model_name':model_name,'hyper_param':p,'test_overall_metric':test_overall_metric,'test_recall':test_recall,'blc_test_recall':blc_test_recall,'valid_recall':valid_recall,'train_accuracy':train_accuracy,'valid_accuracy':valid_accuracy,'test_accuracy':test_accuracy,
                         'train_epoch_loss':train_epoch_loss,'train_epoch_acc':train_epoch_acc,'valid_epoch_loss':valid_epoch_loss,'valid_epoch_acc':valid_epoch_acc,'test_epoch_acc':test_epoch_acc,'test_epoch_loss':test_epoch_loss}, name='')
    Hyper_Details = Hyper_Details.append(new_row)
    m_details = pd.DataFrame()
    m_details = m_details.append(new_row)




    if save_results:
      # Hyper_Details.to_csv(G_path +'/Saved/Grid_Search_Results/' + export_name + date_hour+ '.csv')
      
      pickle.dump(m_details, open(G_path +'/08_Saved_Models_Outpus/Grid_Search_Results/' + model_name, 'wb'))
      pickle.dump(Hyper_Details, open(G_path +'/08_Saved_Models_Outpus/Grid_Search_Results/' + 'Grid_All_' + export_name + date_hour , 'wb'))

    if save_models:
      pickle.dump(model_, open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  model_name, 'wb'))

    Best_Grid_Model = pickle.load(open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  'Best_Grid_Model' +str(i) +'_' + date_hour, 'rb'))

Hyper_Details.sort_values('test_overall_metric', ascending=False,inplace=True)
best_param = Hyper_Details['hyper_param'][0]
best_params = Hyper_Details['hyper_param'][:4]

# Grid_Details = Hyper_Details

# Model_Grid = Model_Training_with_loader(**best_param,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=best_param)
# np.random.seed(0)
# random.seed(0)
# torch.manual_seed(0)
# Model_Grid.train()
# print('Max Avg. Recall on Test ====>> :')
# Grid_Details['test_recall'][:5], Grid_Details['hyper_param'][0]

  0%|          | 0/2 [00:00<?, ?it/s]

[Epoch: 1]  , Train_loss: 1.9 , Train_Acc: 17.3%, Val_loss: 1.8 , Val_Acc: 41.7%, Test_Acc: 36.8%  , run time: 6.1
[Epoch: 2]  , Train_loss: 1.8 , Train_Acc: 26.4%, Val_loss: 1.7 , Val_Acc: 40.0%, Test_Acc: 35.0%  , run time: 6.02


Max_Test_Recall: 27.3%, Max_BLC_Test_Recall: 25.7%, Max_Tr_Acc: 26.4%, Max_V_Acc: 40.0%, Max_Te_Acc: 35.1% :  50%|█████     | 1/2 [00:20<00:20, 20.31s/it, Train_Acc: 26.4% , Valide_Acc: 40.0% , Test_Acc: 35.1%]

[Epoch: 1]  , Train_loss: 1.9 , Train_Acc: 16.6%, Val_loss: 1.8 , Val_Acc: 43.6%, Test_Acc: 39.0%  , run time: 6.59
[Epoch: 2]  , Train_loss: 1.8 , Train_Acc: 24.6%, Val_loss: 1.7 , Val_Acc: 40.7%, Test_Acc: 36.5%  , run time: 6.62


Max_Test_Recall: 27.5%, Max_BLC_Test_Recall: 27.6%, Max_Tr_Acc: 26.4%, Max_V_Acc: 40.8%, Max_Te_Acc: 36.6% : 100%|██████████| 2/2 [00:42<00:00, 21.07s/it, Train_Acc: 24.6% , Valide_Acc: 40.8% , Test_Acc: 36.6%]


In [30]:
Model_Training_with_loader()

TypeError: ignored

In [14]:
print_epochs = True
save_results = False
save_models = False
cross_valid = False
nr_repeat =1
export_name = 'CNN_280x210_MultiClass' # {'280_210_','AutoEncoder_'}
kflods = sk.model_selection.KFold(n_splits=10, shuffle=True)

NNs_grid_searc_cross_valid_trainer(params, X_train ,Y_train, X_valid, Y_valid, X_test, Y_test, kflods, print_epochs, save_results, save_models, cross_valid, nr_repeat, export_name)



NameError: ignored

In [13]:
def NNs_grid_searc_cross_valid_trainer(params, X_train ,Y_train, X_valid, Y_valid, X_test, Y_test, kflods, print_epochs, save_results, save_models, cross_valid, nr_repeat, export_name):

    Param_Details = pd.DataFrame(columns=['model_name','hyper_param','test_accuracy','valid_accuracy','train_accuracy','test_specificity','valid_specificity'
                                      ,'train_epoch_loss','train_epoch_acc','valid_epoch_loss','valid_epoch_acc','test_epoch_acc','test_epoch_loss','test_fpr','test_tpr','test_roc_auc','train_index','valid_index'])
    date_hour = datetime.now().strftime("%d_%b%y_%H-%M") 
    Max_train_Acc = 0  
    Max_valid_Acc = 0 
    Max_test_Acc = 0                  
    Max_test_Recall = 0
    Max_BLC_Test_Recall = 0
    Max_test_overall_metric = 0

    if cross_valid:
        total_iter = nr_repeat*len(params)*kflods.get_n_splits()
    else: total_iter = nr_repeat*len(params)
    iter = 1

    for i in range(nr_repeat):
        for p in params:
            test_accuracy = []
            test_fscore = []
            test_precision = []
            test_recall = []
            train_index = None
            valid_index = None
            if cross_valid:
                training_data = torch.cat([X_train,X_valid],dim=0)
                training_label = torch.cat([Y_train,Y_valid],dim=0)
                
                for fold, (train_index, valid_index) in enumerate(kflods.split(training_data,training_label)):
                    progressbar(iter,total_iter, "Interations: ", 100)
                    iter +=1
                    x_train_ = training_data[train_index]
                    y_train_ = training_label[train_index]
                    x_valid_ = training_data[valid_index]
                    y_valid_ = training_label[valid_index]
                    
                    model_name =  export_name + 'Model' + str(iter) + '_' + date_hour
                    torch.cuda.empty_cache()
                    Model_ = Model_Training_with_loader(**p,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset, testDataset=testDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=p, device=device)
                    np.random.seed(0)
                    random.seed(0)
                    torch.manual_seed(0)
                    torch.cuda.manual_seed(0)
                    torch.backends.cudnn.deterministic = True
                    Model_.train()

                    train_accuracy = max(Model_.Epochs_Train_Acc)
                    train_epoch_loss = Model_.Epochs_Train_loss
                    train_epoch_acc = Model_.Epochs_Train_Acc
                    valid_epoch_loss = Model_.Epochs_Val_loss
                    valid_epoch_acc = Model_.Epochs_Val_Acc
                    test_epoch_loss = Model_.Epochs_test_loss
                    test_epoch_acc = Model_.Epochs_test_Acc

                    model_ = Model_.model.eval().to('cpu')
                    # test Performance
                    X = X_test
                    Y = Y_test
                    Y_OneH = sk.preprocessing.label_binarize(Y,classes=np.unique(Y))
                    Y_pred_prob = model_.forward_noDrop(X)
                    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
                    per_ = recall_specificity(Y,Y_pred).loc['Weighted Avg.']
                    test_accuracy , test_specificity = per_['Recall_Sensitivity'] , per_['Specificity']
                    test_fpr, test_tpr, test_roc_auc = fpr_tpr_score(Y_OneH,Y_pred_prob)

                    # Valid performance
                    X = X_valid
                    Y = Y_valid
                    Y_OneH = sk.preprocessing.label_binarize(Y,classes=np.unique(Y))
                    Y_pred_prob = model_.forward_noDrop(X)
                    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
                    per_ = recall_specificity(Y,Y_pred).loc['Weighted Avg.']
                    valid_accuracy , valid_specificity = per_['Recall_Sensitivity'] , per_['Specificity']

                    new_row = pd.Series({'model_name':model_name,'hyper_param':p,'test_accuracy':test_accuracy,'valid_accuracy':valid_accuracy,'train_accuracy':train_accuracy,'test_specificity':test_specificity,'valid_specificity':valid_specificity
                                      ,'train_epoch_loss':train_epoch_loss,'train_epoch_acc':train_epoch_acc,'valid_epoch_loss':valid_epoch_loss,'valid_epoch_acc':valid_epoch_acc,'test_epoch_acc':test_epoch_acc,'test_epoch_loss':test_epoch_loss
                                      ,'test_fpr':test_fpr,'test_tpr':test_tpr,'test_roc_auc':test_roc_auc,'train_index':train_index,'valid_index':valid_index}, name='')
                    Param_Details = Param_Details.append(new_row)
            else:
                    progressbar(iter,total_iter, "Interations: ", 100)
                    iter +=1
                    x_train_ = X_train
                    y_train_ = Y_train
                    x_valid_ = X_valid
                    y_valid_ = Y_valid
                    
                    model_name =  export_name + 'Model' + str(iter) + '_' + date_hour
                    torch.cuda.empty_cache()
                    Model_ = Model_Training_with_loader(**p,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset, testDataset=testDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=p, device=device)
                    np.random.seed(0)
                    random.seed(0)
                    torch.manual_seed(0)
                    torch.cuda.manual_seed(0)
                    torch.backends.cudnn.deterministic = True
                    Model_.train()

                    train_accuracy = max(Model_.Epochs_Train_Acc)
                    train_epoch_loss = Model_.Epochs_Train_loss
                    train_epoch_acc = Model_.Epochs_Train_Acc
                    valid_epoch_loss = Model_.Epochs_Val_loss
                    valid_epoch_acc = Model_.Epochs_Val_Acc
                    test_epoch_loss = Model_.Epochs_test_loss
                    test_epoch_acc = Model_.Epochs_test_Acc

                    model_ = Model_.model.eval().to('cpu')
                    # test Performance
                    X = X_test
                    Y = Y_test
                    Y_OneH = sk.preprocessing.label_binarize(Y,classes=np.unique(Y))
                    Y_pred_prob = model_.forward_noDrop(X)
                    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
                    per_ = recall_specificity(Y,Y_pred).loc['Weighted Avg.']
                    test_accuracy , test_specificity = per_['Recall_Sensitivity'] , per_['Specificity']
                    test_fpr, test_tpr, test_roc_auc = fpr_tpr_score(Y_OneH,Y_pred_prob)

                    # Valid performance
                    X = X_valid
                    Y = Y_valid
                    Y_OneH = sk.preprocessing.label_binarize(Y,classes=np.unique(Y))
                    Y_pred_prob = model_.forward_noDrop(X)
                    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
                    per_ = recall_specificity(Y,Y_pred).loc['Weighted Avg.']
                    valid_accuracy , valid_specificity = per_['Recall_Sensitivity'] , per_['Specificity']

                    new_row = pd.Series({'model_name':model_name,'hyper_param':p,'test_accuracy':test_accuracy,'valid_accuracy':valid_accuracy,'train_accuracy':train_accuracy,'test_specificity':test_specificity,'valid_specificity':valid_specificity
                                      ,'train_epoch_loss':train_epoch_loss,'train_epoch_acc':train_epoch_acc,'valid_epoch_loss':valid_epoch_loss,'valid_epoch_acc':valid_epoch_acc
                                      ,'test_epoch_acc':test_epoch_acc,'test_epoch_loss':test_epoch_loss,'test_fpr':test_fpr,'test_tpr':test_tpr,'test_roc_auc':test_roc_auc,'train_index':train_index,'valid_index':valid_index}, name='')
                    Param_Details = Param_Details.append(new_row)


    # best_one = np.argmax(Param_Details.test_recall_weighed)
    # best_param = Param_Details.iloc[best_one]['hyper_param']
    # best_train_index = Param_Details.iloc[best_one]['train_index'] 
    # best_valid_index = Param_Details.iloc[best_one]['valid_index']  
    # if cross_valid:
    #         x_train_ = training_data[best_train_index]
    #         y_train_ = training_label[best_train_index]
    #         x_valid_ = training_data[best_valid_index]
    #         y_valid_ = training_label[best_valid_index]

    # Best_Model = Model_.set_params(**best_param)
    # Best_Model.fit(x_train_, y_train_)
    # Y_pred = Best_Model.predict(X_test)
    # Param_Details['hyper_param'] = Param_Details['hyper_param'].apply(lambda x: json.dumps(x))
    # print('------- Precision recal %--------')
    # print(sk.metrics.classification_report(Y_test,Y_pred))
    # print('Best param: ' , best_param)
    # return Best_Model, Param_Details

In [ ]:
# model_ = Model_Grid
model_1 = Best_Grid_Model
X = input_data[2]
Y = labels[2]
# X = Evaluation_set[0]
# Y = Evaluation_set[1]
Y_pred = model_1.forward_noDrop(X).argmax(dim=1)
results = confusion_matrix(Y,Y_pred)
# plot_loss_accuracy(model_)
results

# **Cross Validation: Repeat 5 times with 10 Kfold (overall 50 models)**

In [ ]:
from torch.nn.modules import loss

In [ ]:
# K_Folds_test = sk.model_selection.KFold(n_splits=10, shuffle=True)
K_Folds_valid = sk.model_selection.KFold(n_splits=10, shuffle=True)
dat_hour = datetime.now().strftime("%d_%b_%Y_%H_%M")
print_epochs = False

param1 = {
    'Net':CNN_Nets.Net8_b
    ,'Drop':0.2
    ,'LR':1.1e-3
    ,'batch_size':24
    ,'Momentum':0.77
    ,'epochs':100
    ,'patience':20
    ,'weight_decay':1e-3
    ,'loss_func' : 
    loss.CrossEntropyLoss
    ,'opt_func' : 
    torch.optim.SGD
}

CV_data = torch.cat([input_data[0],input_data[1]],dim=0)
CV_label = torch.cat([labels[0],labels[1]],dim=0)
X_test = input_data[2]
Y_test = labels[2]


CV_Details = pd.DataFrame(columns=['hyper_param','train_recall_weighed','valid_recall_weighed','test_recall_weighed'
                                  ,'valid_recall_simple','test_recall_simple'
                                  ,'valid_specificity_weighed','test_specificity_weighed'
                                  ,'valid_specificity_simple','test_specificity_simple'])

Max_train_Acc = 0  
Max_valid_Acc = 0 
Max_test_Acc = 0                  

pbar1 = tqdm(range(5))
for i in pbar1:
    pbar2 = tqdm(enumerate(K_Folds_valid.split(CV_data)), total=K_Folds_valid.get_n_splits())
    for fold, (training_index, valid_index) in pbar2:
        X_train = CV_data[training_index]
        Y_train = CV_label[training_index]
        X_valid = CV_data[valid_index]
        Y_valid = CV_label[valid_index]

        label_freq = np.bincount(Y_train)
        labels_weights = 1. / label_freq
        weights = labels_weights[Y_train]
        w_sampler = WeightedRandomSampler(weights, len(weights))

        trainDataset = dset(X_train, Y_train)
        validDataset = dset(X_valid, Y_valid)
        testDataset = dset(X_test, Y_test)

        Model_ = Model_Training_with_loader(**param1,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=param1)
        Model_.train()

        model_ = Model_.model.eval().to('cpu')
            
        # Train
        train_recall_weighed = mean(Model_.Epochs_Train_Acc)

        # Valid
        Y_pred = model_(X_valid).argmax(axis=1)
        Y = Y_valid
        valid_recall_weighed, valid_specificity_weighed, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=True)
        valid_recall_simple, valid_specificity_simple, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=False)

        # Test
        Y_pred = model_(X_test).argmax(axis=1)
        Y = Y_test
        test_recall_weighed, test_specificity_weighed, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=True)
        test_recall_simple, test_specificity_simple, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=False)


        if train_recall_weighed > Max_train_Acc: Max_train_Acc = train_recall_weighed
        if valid_recall_weighed > Max_valid_Acc: Max_valid_Acc = valid_recall_weighed
        if test_recall_weighed > Max_test_Acc: Max_test_Acc = test_recall_weighed
        new_row = pd.Series({'train_recall_weighed':train_recall_weighed,'valid_recall_weighed':valid_recall_weighed,'test_recall_weighed':test_recall_weighed
                                      ,'valid_recall_simple':valid_recall_simple,'test_recall_simple':test_recall_simple
                                      ,'valid_specificity_weighed':valid_specificity_weighed,'test_specificity_weighed':test_specificity_weighed
                                      ,'valid_specificity_simple':valid_specificity_simple,'test_specificity_simple':test_specificity_simple}, name='')
        CV_Details = CV_Details.append(new_row)
        pbar1.set_description(f'Max_train_Acc: {Max_train_Acc:.1%}, Max_valid_Acc: {Max_valid_Acc:.1%}, Max_test_Acc: {Max_test_Acc:.1%} ')
        pbar1.set_postfix_str(f'Train_Acc: {train_recall_weighed:.1%} , Valide_Acc: {valid_recall_weighed:.1%} , Test_Acc: {test_recall_weighed:.1%}')


        pickle.dump(CV_Details, open(G_path + '/08_Saved_Models_Outpus/Cross_Valid_Results/CV_280_MultiClass_'+ dat_hour,'wb'))

In [ ]:
col_recall = ['test_recall_weighed','valid_recall_weighed']
model_recalls = CV_Details[col_recall]
col_specificity = ['test_specificity_weighed','valid_specificity_weighed']
model_specificity = CV_Details[col_specificity]

In [ ]:
plt.subplots(figsize=(8,4))
sns.boxplot(data=model_recalls)
plt.title('Test performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()

plt.subplots(figsize=(8,4))
sns.boxplot(data=model_specificity)
plt.title('Valid performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()
